In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data_path = '../../data/raw/Retail_Sales_Data.csv'  # Adjust path if necessary
df = pd.read_csv(data_path)
df.head()



,retailer,store_id,week,product,description,regular_price,competition_1_regular_price,competition_2_regular_price,competition_3_regular_price,competition_4_regular_price,...,competition_4_promo_price,competition_5_promo_price,competition_6_promo_price,competition_7_promo_price,competition_8_promo_price,competition_9_promo_price,competition_10_promo_price,value,volume,quantity
0,retail1,1,1,MintyFresh Mint 18g,MintyFresh refresh mint 18grams,10.308645,10.624815,10.013451,10.378847,9.910385,...,8.500521,7.139537,7.606831,8.339536,7.184384,5.759620,7.376553,85.368310,3.953882,9.707949
1,retail1,1,1,ChocoDelight Dark 200g,200g ChocoDelight smooth,7.947956,7.748726,7.589198,8.085262,7.602363,...,5.766135,4.267236,5.912718,5.156558,6.976287,5.884786,5.335849,82.924731,2.991418,14.922586
2,retail1,1,1,ChocoDelight White 350g,350g ChocoDelight delicious,13.439368,13.751508,13.183621,13.875440,12.982707,...,8.195118,8.214548,8.995387,10.266603,10.940801,8.727031,7.915147,52.527762,6.513600,12.889607
3,retail1,1,1,NuttyCream Hazelnuts 80g,delightful Hazelnuts 80g NuttyCream,5.112777,5.279713,5.222118,5.117789,4.877549,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.373782,5.489587,14.374834
4,retail1,1,1,DarkDream Dark 60g,A decadent DarkDream dark chocolate ice cream ...,14.085425,14.153721,13.860002,14.339698,14.365517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.718252,9.152050,9.193818


In [2]:
# Check for missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
print("Missing values in each column:\n", missing_values)

# Check for non-numeric data types
print("Data types:\n", df.dtypes)

# Basic summary statistics to understand distributions
print("Summary statistics:\n", df.describe(include='all'))


Missing values in each column:
 competition_10_promo_price      8194
competition_9_promo_price       8194
competition_8_promo_price       8194
competition_7_promo_price       8194
competition_6_promo_price       8194
competition_5_promo_price       8194
competition_4_promo_price       8194
competition_3_promo_price       8194
competition_2_promo_price       8194
competition_1_promo_price       8194
promo_price                     8194
retailer                           0
store_id                           0
volume                             0
value                              0
competition_10_regular_price       0
competition_9_regular_price        0
competition_8_regular_price        0
competition_7_regular_price        0
competition_6_regular_price        0
competition_5_regular_price        0
competition_4_regular_price        0
competition_3_regular_price        0
competition_2_regular_price        0
competition_1_regular_price        0
regular_price                      0
descri

Promo Active Indicator:

In [3]:
for col in df.filter(like='competition_promo_price').columns:
    df[f'{col}_active'] = (df[col] > 0).astype(int)

Discount Depth:

In [4]:
for i in range(1, 11):  # assuming 10 competitors
    df[f'competition_{i}_discount_depth'] = (
        df[f'competition_{i}_regular_price'] - df[f'competition_{i}_promo_price']
    ).clip(lower=0)

Average Discount Depth:

In [5]:
discount_columns = [f'competition_{i}_discount_depth' for i in range(1, 11)]
df['avg_competition_discount_depth'] = df[discount_columns].mean(axis=1)

Number of Competitors Running Promotions:

In [6]:
promo_active_columns = [f'{col}_active' for col in df.filter(like='competition_promo_price').columns]
df['num_competitors_active_promos'] = df[promo_active_columns].sum(axis=1)


Price Ratio Interaction:

In [7]:
for i in range(1, 11):
    df[f'price_comp_{i}_promo_interaction'] = df['regular_price'] / df[f'competition_{i}_promo_price']


Drop Raw competition_promo_price Columns:

In [15]:
promo_columns = [f'competition_{i}_promo_price' for i in range(1, 11)]
df.drop(columns=promo_columns, inplace=True)


Handle Outliers and Transform Skewed Features

In [10]:
for col in ['quantity', 'value', 'regular_price']:
    upper_limit = df[col].quantile(0.95)
    df[col] = np.where(df[col] > upper_limit, upper_limit, df[col])


In [11]:
df['quantity'] = np.log1p(df['quantity'])
df['value'] = np.log1p(df['value'])

Add Weekly Seasonality:

In [14]:
df['sin_week'] = np.sin(2 * np.pi * df['week'] / 52)
df['cos_week'] = np.cos(2 * np.pi * df['week'] / 52)

Lag Features:

In [12]:
df['quantity_lag_1'] = df['quantity'].shift(1)
df['value_lag_1'] = df['value'].shift(1)
df.fillna(0, inplace=True)  # Fill NaN values created by lagging


In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8840 entries, 0 to 8839
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   retailer                         8840 non-null   object 
 1   store_id                         8840 non-null   int64  
 2   week                             8840 non-null   int64  
 3   product                          8840 non-null   object 
 4   description                      8840 non-null   object 
 5   regular_price                    8840 non-null   float64
 6   competition_1_regular_price      8840 non-null   float64
 7   competition_2_regular_price      8840 non-null   float64
 8   competition_3_regular_price      8840 non-null   float64
 9   competition_4_regular_price      8840 non-null   float64
 10  competition_5_regular_price      8840 non-null   float64
 11  competition_6_regular_price      8840 non-null   float64
 12  competition_7_regula

In [17]:
print(df.columns)

Index(['retailer', 'store_id', 'week', 'product', 'description',
       'regular_price', 'competition_1_regular_price',
       'competition_2_regular_price', 'competition_3_regular_price',
       'competition_4_regular_price', 'competition_5_regular_price',
       'competition_6_regular_price', 'competition_7_regular_price',
       'competition_8_regular_price', 'competition_9_regular_price',
       'competition_10_regular_price', 'promo_price', 'value', 'volume',
       'quantity', 'competition_1_discount_depth',
       'competition_2_discount_depth', 'competition_3_discount_depth',
       'competition_4_discount_depth', 'competition_5_discount_depth',
       'competition_6_discount_depth', 'competition_7_discount_depth',
       'competition_8_discount_depth', 'competition_9_discount_depth',
       'competition_10_discount_depth', 'avg_competition_discount_depth',
       'num_competitors_active_promos', 'price_comp_1_promo_interaction',
       'price_comp_2_promo_interaction', 'price_

In [18]:
missing_values = df.isnull().sum().sort_values(ascending=False)
print("Missing values in each column:\n", missing_values)

# Check for non-numeric data types
print("Data types:\n", df.dtypes)

# Basic summary statistics to understand distributions
print("Summary statistics:\n", df.describe(include='all'))

Missing values in each column:
 retailer                           0
price_comp_3_promo_interaction     0
competition_6_discount_depth       0
competition_7_discount_depth       0
competition_8_discount_depth       0
competition_9_discount_depth       0
competition_10_discount_depth      0
avg_competition_discount_depth     0
num_competitors_active_promos      0
price_comp_1_promo_interaction     0
price_comp_2_promo_interaction     0
price_comp_4_promo_interaction     0
store_id                           0
price_comp_5_promo_interaction     0
price_comp_6_promo_interaction     0
price_comp_7_promo_interaction     0
price_comp_8_promo_interaction     0
price_comp_9_promo_interaction     0
price_comp_10_promo_interaction    0
quantity_lag_1                     0
value_lag_1                        0
sin_week                           0
competition_5_discount_depth       0
competition_4_discount_depth       0
competition_3_discount_depth       0
competition_2_discount_depth       0
week  

In [19]:
# Drop `promo_price` as it hasn't been transformed or engineered further
df.drop(columns=['promo_price'], inplace=True)

# Optionally, drop `competition_X_regular_price` columns if they’re no longer needed
regular_price_columns = [f'competition_{i}_regular_price' for i in range(1, 11)]
df.drop(columns=regular_price_columns, inplace=True)

In [20]:
print(df.columns)

Index(['retailer', 'store_id', 'week', 'product', 'description',
       'regular_price', 'value', 'volume', 'quantity',
       'competition_1_discount_depth', 'competition_2_discount_depth',
       'competition_3_discount_depth', 'competition_4_discount_depth',
       'competition_5_discount_depth', 'competition_6_discount_depth',
       'competition_7_discount_depth', 'competition_8_discount_depth',
       'competition_9_discount_depth', 'competition_10_discount_depth',
       'avg_competition_discount_depth', 'num_competitors_active_promos',
       'price_comp_1_promo_interaction', 'price_comp_2_promo_interaction',
       'price_comp_3_promo_interaction', 'price_comp_4_promo_interaction',
       'price_comp_5_promo_interaction', 'price_comp_6_promo_interaction',
       'price_comp_7_promo_interaction', 'price_comp_8_promo_interaction',
       'price_comp_9_promo_interaction', 'price_comp_10_promo_interaction',
       'quantity_lag_1', 'value_lag_1', 'sin_week', 'cos_week'],
      dty

In [21]:

df.to_csv("../../data/processed/processed_data.csv", index=False)
